## PyMC3

### State Space Model:
https://gist.github.com/npyoung/adc097f95c6148a5e31c2f388efaa697

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano
import theano.tensor as tt
import pdb

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.8


In [3]:
theano.config.compute_test_value = 'ignore'

\begin{align}
    y_{t} &= F_{t}x_{t} + X_{t}\beta_{t} + \nu_{t} \\
    x_{t} &= G_{t}x_{t-1} + Z_{t}\gamma_{t} + \omega_{t} \\
    \nu_{t} &\sim \mathcal{N}(0, V_{t}) \\
    \omega_{t} &\sim \mathcal{N}(0, W_{t}) \\
\end{align}

For simplicity we well assume that $X_{t} = 0$, $Z_{t}= 0$, $F_{t}=F\; \forall t\in 1, ..., T$,  $G_{t}=G\; \forall t\in 1, ..., T$, $V_{t}=V\; \forall t\in 1, ..., T$, $W_{t}=W\; \forall t\in 1, ..., T$

In [4]:
F = np.array([[1, 0]])
G = np.array([[1, 1], [0, 1]])

# V
psiy = 10
V = np.array([[1/psiy]])
chol_V = np.linalg.cholesky(V)

# W
psi1 = 10
psi2 = 10
W = np.array([[1/psi1, 0], [0, 1/psi2]])
chol_W = np.linalg.cholesky(W)

# Sample Size
T = 100

# inital state
x0 = np.zeros(2)
x = np.zeros((T, 2))
x[0] = x0
y = np.zeros((T,1))

for t in range(1, T):
    x[t] = np.dot(G, x[t-1].T).T+np.dot(chol_W,  np.random.randn(chol_W.shape[0]))
    y[t] = np.dot(F, x[t].T).T+np.dot(chol_V,  np.random.randn(chol_V.shape[0]))

plt.plot(y)

In [7]:
from pymc3 import MvNormal, Normal, Flat, Continuous

class StateSpaceModel(Continuous):
    """
    A state space model with Gaussian noise.
    
    This models only the state variables so that the form of the observation
    noise can be specified separately.
    
    Parameters
    ----------
    V : tensor
        V = p.s.d , observation cov-matrix
    chol_V : tensor
        cholesky decomposition of V
    W : tensor
        W = p.s.d , state cov-matrix
    chol_W : tensor
        cholesky decomposition of W
    G : tensor
        state update matrix
    F : tensor
        input matrix
    u : tensor
        (time x dim), inputs to the system
    init : distribution
        distribution for initial value (defaults to Flat())
    """
    def __init__(self, V=None, chol_V=None, W=None, chol_W=None, F=None, G=None, X=None, Z=None, beta=None, gamma=None, init=Flat.dist(), *args, **kwargs):
        super(StateSpaceModel, self).__init__(*args, **kwargs)
        
        if chol_V is not None:
            self.chol_V = chol_V
        else: 
            self.chol_V = np.linalg.cholesky(V)
        
        if chol_W is not None:
            self.chol_W = chol_W
        else:
            self.chol_W = np.linalg.cholesky(W)
        
        self.F = F
        self.G = G
        self.X = X
        self.Z = Z
        self.beta = beta
        self.gamma = gamma
        self.init = init
        self.mean = 0.
        
            
#    def random(self, point=None, size=None):
#        tau, sd, A, B, u, init = draw_values([self.tau, self.sd, self.A, self.B, self.u, self.init], point=point)
#        
#        
#        chol_V = self.chol_V,
#        F = self.F
#        G = self.G
#        X = self.X
#        Z = self.Z
#        beta = self.beta
#        gamma = self.gamma
#        init = self.init
#
#        T, D = size
#        x = np.zeros(size)
#        y = np.zeros((T, 1))
#        x[0,:] = init
#        
#        for t in range(1, T):
#            x[t] = np.dot(G, x[t-1].T).T + np.dot(W_chol,  np.random.randn(W_chol.shape[0]))
#            y[t] = np.dot(F, x[t].T).T + np.dot(V_chol,  np.random.randn(V_chol.shape[0]))
#
#        return x, y
    
    
    def logp(self, x):
        V = np.dot(self.chol_V, self.chol_V.T)
        W = np.dot(self.chol_W, self.chol_W.T)
        chol_W = self.chol_W
        F = self.F
        G = self.G
        init = self.init
        
        x_im1 = x[:-1]
        x_i = x[1:]

        innov_like = MvNormal.dist(mu=tt.dot(G, x_im1.T), chol=chol_W).logp(x_i.T)
        
        return tt.sum(init.logp(x[0])) + tt.sum(innov_like)
    

In [10]:
x.shape

(100, 2)

In [9]:
obs = x
with pm.Model() as model:
    F = np.array([[1, 0]])
    G = np.array([[1, 1], [0, 1]])
    psiy = pm.Gamma('psiy', alpha=1e-3, beta=1e-3)
    chol_V = np.array([[1/psiy]])
    sd_dist = pm.HalfCauchy.dist(beta=2.5)
    chol_W = pm.LKJCholeskyCov('chol_W', eta=20.0, n=2, sd_dist=sd_dist)
    cW = pm.expand_packed_triangular(2, chol_W, lower=True)

    X = StateSpaceModel('x', chol_V=chol_V, chol_W=cW, F=F, G=G, init=x0, observed=obs)
    
    trace = pm.sample(10)

/usr/local/lib/python3.6/dist-packages/theano/tensor/subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
/usr/local/lib/python3.6/dist-packages/theano/tensor/subtensor.py:2339: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out[0][inputs[2:]] = inputs[1]


ValueError: incompatible dimensions

In [ ]:
mod = StateSpaceModel(chol_V=chol_V, chol_W=chol_W, F=F, G=G, init=Flat.dist())

In [ ]:
with pm.Model() as m:
    x = pm.Normal('x', mu=0., sigma=1.)


m = pm.Model()
x = m.Var('x', pm.Normal.dist(mu=0., sigma=1.))


print(type(x))                              # ==> <class 'pymc3.model.FreeRV'>
print(m.free_RVs)                           # ==> [x]
print(x.distribution.logp(5.))              # ==> Elemwise{switch,no_inplace}.0
print(x.distribution.logp(5.).eval({}))     # ==> -13.418938533204672
print(m.logp({'x': 5.}))   

In [ ]:
MvNormal.dist(mu=tt.dot(G, x_im1.T), chol=np.kron(np.eye(99), chol_W)).logp(x_i.T)

In [ ]:
print(tt.dot(G, x_im1[0].T))

In [ ]:
print(x_im1.T.shape)
print(x_i.T.shape)

In [ ]:
np.array([[1, 2], [3, 4]])

In [ ]:
np.kron(np.eye(2), np.array([[1, 2], [3, 4]]))

In [ ]:
MvNormal.dist(mu=tt.dot(G, x_im1[0:2].T), chol=chol_W).logp(x_i[0:2].T)

In [ ]:
tt.dot(G, x[3,:])

In [ ]:
with pm.Model() as model:
    F = np.array([[1, 0]])
    G = np.array([[1, 1], [0, 1]])
    psiy = pm.Gamma('psiy', alpha=1e-3, beta=1e-3)
    chol_V = np.array([[1/psiy]])
    
    trace = mc.sample(1000)

In [ ]:
pm.traceplot(trace, compact=False)

In [ ]:
with pm.Model() as model:
    F = np.array([[1, 0]])
    G = np.array([[1, 1], [0, 1]])
    psiy = pm.Gamma('psiy', alpha=1e-3, beta=1e-3)
    chol_V = np.array([[1/psiy]])
    sd_dist = pm.HalfCauchy.dist(beta=2.5)
    chol_W = pm.LKJCholeskyCov('chol_W', eta=20.0, n=2, sd_dist=sd_dist)
    cW = pm.expand_packed_triangular(2, chol_W, lower=True)
    
    X = StateSpaceModel('x', chol_V=chol_V, chol_W=cW, F=F, G=G, init=x0, observed=x)
    
    trace = pm.sample(1000)

In [ ]:
pm.traceplot(trace, compact=False)

In [ ]:
trace['A'].mean(axis=0)



In [ ]:
true_A

In [ ]:
true_sig_o = 0.2
true_tau_o = 1./true_sig_o**2
true_C = np.random.randn(1, 2)
y = np.random.randn(x.shape[0], true_C.shape[0]) * true_sig_o + np.dot(true_C, x.T).T

print("True C: \n{}".format(true_C))
print("True tau_o: {}".format(true_tau_o))
plt.plot(y)

In [ ]:
with mc.Model() as model:
    A = mc.Normal('A', mu=np.eye(2), tau=1e-5, shape=(2,2))
    Tau = mc.Gamma('tau', mu=100, sd=100)
    
    X = StateSpaceModel('x', A=A, B=T.zeros((1,1)), u=T.zeros((x.shape[0],1)), tau=Tau, shape=(y.shape[0], 2))
    
    C = mc.Normal('C', mu=np.zeros((1,2)), tau=1e-5, shape=(1,2))
    Tau_o = mc.Gamma('tau_o', mu=100, sd=100)
    
    Y = mc.Normal('y', mu=T.dot(C, X.T).T, tau=Tau_o, observed=y)
    
    trace = mc.sample(10000, start=mc.find_MAP())

In [ ]:
n=2000
A = np.random.randn(n, n)
B = np.random.randn(n, n)

In [ ]:
start_time = time.time()
A.dot(B)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Generate data for a simple 1 dimensional example problem
n1_c = 300; n2_c = 300; n3_c = 300
cluster1 = np.random.randn(n1_c) + -1
cluster2 = np.random.randn(n2_c) + 0
cluster3 = np.random.randn(n3_c) + 2

x = np.concatenate((cluster1, cluster2, cluster3))
y = np.concatenate((1*np.ones(n1_c),
                    2*np.ones(n2_c),
                    3*np.ones(n3_c))) - 1

In [ ]:
# Ordered logistic regression
with pm.Model() as model:
    cutpoints = pm.Normal("cutpoints", mu=[-1,1], sigma=10, shape=2,
                          transform=pm.distributions.transforms.ordered)
    y_ = pm.OrderedLogistic("y", cutpoints=cutpoints, eta=x, observed=y)
    tr = pm.sample(1000)

In [ ]:
# Plot the results
plt.hist(cluster1, 30, alpha=0.5);
plt.hist(cluster2, 30, alpha=0.5);
plt.hist(cluster3, 30, alpha=0.5);
plt.hist(tr["cutpoints"][:,0], 80, alpha=0.2, color='k');
plt.hist(tr["cutpoints"][:,1], 80, alpha=0.2, color='k');